### Домашняя работа №8

1. Можно ли отобрать наиболее значимые признаки с помощью PCA?
2. Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()
3. Обучить любую модель классификации на датасете iris до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

#### 1. Можно ли отобрать наиболее значимые признаки с помощью PCA?

Нельзя, т.к. отбор признаков и метод главных компонент - это разные методы решения задачи понижения размерности данных. При создании новых признаков меньшей размерности $Z=X\cdot W$, через линейную комбинацию, метод главных компонент задействует все исходные признаки $X$.

#### 2. Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# импортируем наше дерево из Lesson_8.py
from Lesson_8 import *

%matplotlib inline

In [2]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
y = iris.target

X.shape

(150, 4)

In [3]:
means = X.mean(axis=0)
stds = X.std(axis=0, ddof=1)

X_scaled = (X - means)/stds

In [4]:
def fro_norm(X):
    return np.sum(X ** 2) ** 0.5

In [5]:
def pca_with_svd(X, n_components):
    U, s, Vt = np.linalg.svd(X)
    
    D = np.zeros_like(X, dtype=float)
    D[np.diag_indices(min(X.shape))] = s
    V = Vt.T
    
    # проверка
    np.testing.assert_array_almost_equal(U.dot(D).dot(Vt), X)
    
    W = V[:, :n_components]
    Z = X.dot(W)
    
    return Z

In [6]:
X_pca_4 = pca_with_svd(X_scaled, n_components=4)
X_pca_3 = pca_with_svd(X_scaled, n_components=3)
X_pca_2 = pca_with_svd(X_scaled, n_components=2)
X_pca_1 = pca_with_svd(X_scaled, n_components=1)

In [7]:
print('Норма Фробениуса:')
print(f' - исходной матрицы X:   {fro_norm(X_scaled)}')
print(f' - четырех ГК матрицы X: {fro_norm(X_pca_4)}')
print(f' - трех ГК матрицы X:    {fro_norm(X_pca_3)}')
print(f' - двух ГК матрицы X:    {fro_norm(X_pca_2)}')
print(f' - одной ГК матрицы X:   {fro_norm(X_pca_1)}')

Норма Фробениуса:
 - исходной матрицы X:   24.413111231467408
 - четырех ГК матрицы X: 24.413111231467404
 - трех ГК матрицы X:    24.349814976137615
 - двух ГК матрицы X:    23.89658374981683
 - одной ГК матрицы X:   20.85320538102637


Норма Фробениуса существенно изменяется с понижением количества главных компонент.

#### 3. Обучить любую модель классификации на датасете iris до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [8]:
def accuracy_score(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [9]:
splitted_data = train_test_split(X_scaled, y, test_size = 0.35, random_state = 42)
splitted_data_pca = train_test_split(X_pca_2, y, test_size = 0.35, random_state = 42)

In [10]:
def classify_irises_with_tree(*data):
    X_train, X_test, y_train, y_test = data
    
    # достаточно двух разбиений
    tree = build_tree(X_train, y_train, max_depth=2)
    
    train_pred = predict(X_train, tree)
    test_pred = predict(X_test, tree)
    
    print('Доля верных ответов:')
    print(f' - на обучающей выборке: {accuracy_score(y_train, train_pred)}')
    print(f' - на тестовой выборке:  {accuracy_score(y_test, test_pred)}')

In [11]:
classify_irises_with_tree(*splitted_data)

Доля верных ответов:
 - на обучающей выборке: 94.84536082474226
 - на тестовой выборке:  98.11320754716981


In [12]:
classify_irises_with_tree(*splitted_data_pca)

Доля верных ответов:
 - на обучающей выборке: 92.78350515463917
 - на тестовой выборке:  94.33962264150944
